In [13]:
import cv2

# Initialize HOG person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

video_name = '/Users/tracyhanwenyu/Desktop/Dataset/VA1.mp4'
cap = cv2.VideoCapture(video_name)

time_skips = float(1000) #skip every 1 seconds. modify if need

count = 0
success,image = cap.read()
while success:
        
    # Detect people in the image
    boxes, weights = hog.detectMultiScale(image, winStride=(4, 4), padding=(8, 8), scale=1.05)

    # Draw rectangle around each person
    for (x, y, w, h) in boxes:
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
    
    # save image
    cv2.imwrite("/Users/tracyhanwenyu/Desktop/images/frame%d.jpg" % count, image)
    
    # Count the number of people detected
    num_people = len(boxes)
    print('Number of people detected:', num_people)
    
    cap.set(cv2.CAP_PROP_POS_MSEC, (count*time_skips))
    print('New frame captured: ' + str(count))

    # move the time
    success,image = cap.read()
    count += 1

# release after reading    
cap.release()

Number of people detected: 9
New frame captured: 0
Number of people detected: 9
New frame captured: 1
Number of people detected: 8
New frame captured: 2
Number of people detected: 7
New frame captured: 3
Number of people detected: 7
New frame captured: 4
Number of people detected: 11
New frame captured: 5
Number of people detected: 11
New frame captured: 6
Number of people detected: 8
New frame captured: 7
Number of people detected: 3
New frame captured: 8
Number of people detected: 5
New frame captured: 9
Number of people detected: 6
New frame captured: 10
Number of people detected: 8
New frame captured: 11
Number of people detected: 3
New frame captured: 12
Number of people detected: 6
New frame captured: 13
Number of people detected: 7
New frame captured: 14
Number of people detected: 8
New frame captured: 15
Number of people detected: 7
New frame captured: 16
Number of people detected: 5
New frame captured: 17
Number of people detected: 7
New frame captured: 18
Number of people det

In [1]:
%cd /content
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt

from yolov5 import utils
display = utils.notebook_init()

print("if you want to use the default images, just click `Cancel upload`")

!python detect.py --weights yolov5s.pt --img 256 --conf 0.25 --source /Users/tracyhanwenyu/Desktop/images

YOLOv5 🚀 v7.0-306-gb599ae42 Python-3.8.8 torch-2.2.2 CPU


Setup complete ✅ (8 CPUs, 16.0 GB RAM, 210.1/465.6 GB disk)
if you want to use the default images, just click `Cancel upload`
detect: weights=['yolov5s.pt'], source=/Users/tracyhanwenyu/Desktop/images, data=data/coco128.yaml, imgsz=[256, 256], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-306-gb599ae42 Python-3.8.8 torch-2.2.2 CPU

100%|██████████████████████████████████████| 14.1M/14.1M [00:01<00:00, 8.04MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
image 1/25 /Users/tracyhanwenyu/Desktop/images/frame0.jpg: 160x256 3 persons, 64.3ms
image 2/25 /Users/tracyhanwenyu/Desktop/images/frame1.jpg: 160x256 3 p

In [18]:
import torchvision
import cv2
import numpy as np
import torch
import torchvision.transforms as T

# Load the pre-trained Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Load the video
cap = cv2.VideoCapture('/Users/tracyhanwenyu/Desktop/Dataset/VA1.mp4')

# Initialize frame counter
frame_counter = 0
seconds = 0

fps = cap.get(cv2.CAP_PROP_FPS)

while(cap.isOpened()):
    # Set the position in the video to the current second
    cap.set(cv2.CAP_PROP_POS_MSEC, seconds * 1000)
    # Read the next frame
    ret, frame = cap.read()

    if ret == True:
        frame_model = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_model = frame_model / 255.0
        frame_model = np.transpose(frame_model, (2, 0, 1))
        frame_model = torch.from_numpy(frame_model).float()

        # Apply the Faster R-CNN model to the frame
        output = model([frame_model])

         # Apply non-maximum suppression
        nms_indices = torchvision.ops.nms(output[0]['boxes'], output[0]['scores'], 0.3)
        output[0]['boxes'] = output[0]['boxes'][nms_indices]
        output[0]['labels'] = output[0]['labels'][nms_indices]

        # Filter out the detections with low confidence scores
        high_conf_indices = [i for i, score in enumerate(output[0]['scores']) if score > 0.7]
        output[0]['boxes'] = output[0]['boxes'][high_conf_indices]
        output[0]['labels'] = output[0]['labels'][high_conf_indices]

        # Count the number of people detected in the frame
        num_people = sum(1 for box, label in zip(output[0]['boxes'], output[0]['labels']) if label == 1)

        # Draw bounding boxes around the detected people
        for box, label in zip(output[0]['boxes'], output[0]['labels']):
            if label == 1:
                cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)
        
        # Save the frame
        cv2.imwrite(f'/Users/tracyhanwenyu/Desktop/images/frame{frame_counter}.jpg', frame)

        # Increment frame counter
        frame_counter += 1
        seconds += 1

        # Print the number of people detected in the frame along with the frame number
        print(f'Frame {frame_counter}: Number of people: {num_people}')
    else:
        break

# Release the video capture
cap.release()

# Close all OpenCV windows
cv2.destroyAllWindows()


Frame 1: Number of people: 3
Frame 2: Number of people: 5
Frame 3: Number of people: 3
Frame 4: Number of people: 4
Frame 5: Number of people: 4
Frame 6: Number of people: 4
Frame 7: Number of people: 4
Frame 8: Number of people: 1
Frame 9: Number of people: 2
Frame 10: Number of people: 3
Frame 11: Number of people: 3
Frame 12: Number of people: 2
Frame 13: Number of people: 1
Frame 14: Number of people: 2
Frame 15: Number of people: 1
Frame 16: Number of people: 2
Frame 17: Number of people: 3
Frame 18: Number of people: 2
Frame 19: Number of people: 1
Frame 20: Number of people: 2
Frame 21: Number of people: 3
Frame 22: Number of people: 4
Frame 23: Number of people: 4
Frame 24: Number of people: 3
